# Bag of Words (BoW) Model in NLP

Trong notebook này, chúng ta sẽ hiện thực mô hình **Bag of Words (BoW)** để biến đổi văn bản thành vector số. 

Các bước:
- Tiền xử lý văn bản
- Đếm tần suất từ
- Lựa chọn từ phổ biến nhất
- Xây dựng ma trận BoW
- Trực quan hoá bằng biểu đồ và Word Cloud
- Thử ứng dụng trong **Text Classification** (dùng Scikit-learn)

Các thư viện sử dụng: `nltk`, `re`, `heapq`, `matplotlib`, `seaborn`, `numpy`, `pandas`, `wordcloud`, `sklearn`.

In [ ]:
!pip install nltk matplotlib seaborn wordcloud pandas scikit-learn

In [ ]:
import nltk
import re
import heapq
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

nltk.download('punkt')
nltk.download('stopwords')

## Step 1: Preprocessing the Text
Ở đây ta lấy một đoạn văn bản mẫu và tiền xử lý để phục vụ cho BoW.

In [ ]:
text = """Beans. I was trying to explain to somebody as we were flying in, that's corn. That's beans. 
And they were very impressed at my agricultural knowledge. Please give it up for Amaury once again 
for that outstanding introduction. I have a bunch of good friends here today, including somebody who 
I served with, who is one of the finest senators in the country and we're lucky to have him, your Senator, 
Dick Durbin is here. I also noticed, by the way, former Governor Edgar here, who I haven't seen in a long time 
and somehow he has not aged and I have. And it's great to see you, Governor. I want to thank President Killeen 
and everybody at the U of I System for making it possible for me to be here today. And I am deeply honored at 
the Paul Douglas Award that is being given to me. He is somebody who set the path for so much outstanding public 
service here in Illinois. Now, I want to start by addressing the elephant in the room. I know people are still 
wondering why I didn't speak at the commencement."""

dataset = nltk.sent_tokenize(text)

for i in range(len(dataset)):
    dataset[i] = dataset[i].lower()
    dataset[i] = re.sub(r'\W', ' ', dataset[i])
    dataset[i] = re.sub(r'\s+', ' ', dataset[i])

for i, sentence in enumerate(dataset):
    print(f"Sentence {i+1}: {sentence}")

## Step 2: Counting Word Frequencies
Ta đếm tần suất từ và loại bỏ stopwords.

In [ ]:
word2count = {}
for data in dataset:
    words = nltk.word_tokenize(data)
    for word in words:
        word2count[word] = word2count.get(word, 0) + 1

stop_words = set(stopwords.words('english'))
filtered_word2count = {w: c for w, c in word2count.items() if w not in stop_words}

word_freq_df = pd.DataFrame(list(filtered_word2count.items()), columns=['Word','Frequency']).sort_values(by='Frequency', ascending=False)
word_freq_df.head(10)

## Step 3: Selecting Top Frequent Words
Chọn ra 10 từ phổ biến nhất và hiển thị biểu đồ.

In [ ]:
freq_words = heapq.nlargest(10, filtered_word2count, key=filtered_word2count.get)
print("Top 10 frequent words:", freq_words)

top_words = sorted(filtered_word2count.items(), key=lambda x: x[1], reverse=True)[:10]
words, counts = zip(*top_words)

plt.figure(figsize=(10,6))
plt.bar(words, counts, color='skyblue')
plt.title('Top 10 Most Frequent Words')
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()

## Step 4: Building BoW Matrix
Ma trận BoW nhị phân cho từng câu.

In [ ]:
X = []
for data in dataset:
    vector = [1 if word in nltk.word_tokenize(data) else 0 for word in freq_words]
    X.append(vector)
X = np.asarray(X)

plt.figure(figsize=(10,6))
sns.heatmap(X, cmap='RdYlGn', cbar=False, annot=True, fmt="d", xticklabels=freq_words,
            yticklabels=[f"Sentence {i+1}" for i in range(len(dataset))])
plt.title('Bag of Words Matrix')
plt.show()

## Step 5: Word Cloud Visualization
Word Cloud thể hiện trực quan độ phổ biến của từ.

In [ ]:
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(filtered_word2count)
plt.figure(figsize=(10,6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

## Step 6: Application - Text Classification
Áp dụng BoW để phân loại văn bản đơn giản bằng **Naive Bayes**.

Ở đây ta tạo một dataset nhỏ với 2 nhãn: *positive* và *negative*.

In [ ]:
corpus = [
    'I love this movie, it was fantastic and wonderful',
    'This film was horrible and boring',
    'I enjoyed the storyline and the acting',
    'I dislike this movie, it was too slow',
    'Amazing experience, would watch again',
    'Terrible film, waste of time'
]

labels = ['positive','negative','positive','negative','positive','negative']

# Vector hóa bằng CountVectorizer (BoW)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=42)

clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

## Kết luận
Mô hình BoW có thể được dùng để biểu diễn văn bản cho nhiều tác vụ NLP như:
- **Text Classification** (như ví dụ)
- **Sentiment Analysis**
- **Document Clustering**

Tuy nhiên, nó không giữ ngữ cảnh, nên khi cần ngữ nghĩa và mối quan hệ từ, ta nên dùng các mô hình khác như **Word2Vec**, **TF-IDF**, **BERT**,...